In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import glob
import os
import polars as pl
# import duckdb as dd
from tqdm import tqdm
import matplotlib.pyplot as plt
import cv2
from pydicom import dcmread
import warnings
from sklearn.preprocessing import LabelEncoder
import pickle
import gc
import ctypes
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

2024-08-06 05:09:42.137416: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-06 05:09:42.137559: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-06 05:09:42.274410: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
model = keras.models.\
load_model("/kaggle/input/keras_base_lnfn_l1_l2/keras/default/1/keras_base_left_neural_foraminal_narrowing_l1_l2.h5")

In [ ]:
rows = {}
weight_dict = {'normal_mild':1, 'moderate':2, 'severe':4}
conditions = ['spinal_canal_stenosis', 'neural_foraminal_narrowing', 'subarticular_stenosis']
sides = ['left', 'right']
vertebrae_levels = ['l1_l2', 'l2_l3', 'l3_l4', 'l4_l5', 'l5_s1']
severity_levels = ['normal_mild', 'moderate', 'severe']

for c in conditions:
    for v in vertebrae_levels:
        if c != 'spinal_canal_stenosis':
            for s in sides:
                for st in studies:
                    #print(st+'_'+s+'_'+c+'_'+v)
                    rows[st+'_'+s+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])
        else:
            for st in studies:
                #print(st+'_'+c+'_'+v)
                rows[st+'_'+c+'_'+v] = np.array([0.333333 * 2, 0.333333 * 2, 0.333333 * 2])

In [10]:
features = []
dicom_ds = dcmread('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/12.dcm')
img_array = dicom_ds.pixel_array
features.append(np.mean(img_array.T, axis=0))


feature_resized = []
for img in features:
    feature_resized.append(np.resize(img,(128,)))
    
final = np.array(feature_resized)

In [11]:
final.shape

(1, 128)

In [12]:
y_proba = model.predict(final)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 909ms/step


I0000 00:00:1722921523.439021     112 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [19]:
y_proba

array([[3.523812e-02, 9.638161e-01, 9.457982e-04]], dtype=float32)

In [6]:
test_dict = {}
image_files = []

for dirname, _, filenames in os.walk('/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images'):
    for filename in filenames:
        test_dict[os.path.join(dirname, filename).split('/')[-3]] = image_files
        image_files.append(os.path.join(dirname, filename))
        #dicom_ds = dcmread(dcm_image_path)
        #img_array = dicom_ds.pixel_array
        #features.append(np.mean(img_array.T, axis=0))
        # print(os.path.join(dirname, filename))
        # print(os.path.join(dirname, filename).split('/')[-3])

In [12]:
for key, value in test_dict.items():
    print(key)
    for file in value:
        print(file)

44036939
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/12.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/18.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/9.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/22.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/25.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/14.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/11.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/24.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classification/test_images/44036939/3844393089/23.dcm
/kaggle/input/rsna-2024-lumbar-spine-degenerative-classi